In [35]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_boston

# from sklearn.template import skltemplate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from plot_classifier import plot_classifier
#from sklearn.plot_classifier import plot_classifier
%matplotlib inline
import sqlite3

In [52]:
## A local DB mtpj with whole image of Flights  16M rows
database = "mtpj.db"

conn = sqlite3.connect(database)

## For prove of concept use Flights_slim instead of Flights
sql_in="""SELECT cf.delay_pct as carrier_v, 
          --of.delay_pct as origin_v,
          df.delay_pct as dest_v,
          df.airport as dest,
          CAST(fl.crs_arr_hh24  as interger) crs_arr_hh24,
          hf.delay_pct  as                    hh24_v,
          CAST(crs_elapsed_time as interger) crs_elapsed_time,
          CAST(fl.fl_month      as interger) month,
          mf.delay_pct  as                   month_v,
          CAST(haul_type        as interger) haul_type,
          CAST(arr_delay_min_gy as interger) arr_delay_min_gy,
          CAST(arr_delay_min_bin as interger) arr_delay_min_bin,
          CAST(arr_delay_ind    as interger) arr_delay_ind,
          CAST(arr_delay        as interger) arr_delay
          FROM  FLIGHTS_slim fl, CARRIER_DELAY_F cf, AIRPORT_f df, AIRPORT_f of,
                HH24_F      hf, MONTH_F mf
          WHERE fl.mkt_unique_carrier = cf.mkt_unique_carrier
          AND   fl.dest               = df.airport
          AND   fl.origin             = of.airport
          AND   fl.crs_arr_hh24       = hf.crs_arr_hh24
          AND   fl.fl_month           = mf.fl_month
          AND   fl.cancelled          = '0.0'       --  cancelled = No
          AND   CAST(crs_elapsed_time as REAL) > 60
          ORDER BY RANDOM()
          LIMIT 250000;"""

df_stage = pd.read_sql(sql_in, con=conn)
conn.close()
df_stage
##df_stage=pd.read_csv('flights_f1.csv')

,carrier_v,dest_v,dest,crs_arr_hh24,hh24_v,crs_elapsed_time,month,month_v,haul_type,arr_delay_min_gy,arr_delay_min_bin,arr_delay_ind,arr_delay
0,8.33723,44.79845,EWR,11,29.34927,82,1,33.52233,0,-15,-15,0,-26
1,5.37583,35.65979,STS,9,27.26231,102,1,33.52233,0,15,15,1,9
2,11.70175,29.17117,ATL,9,27.26231,185,1,33.52233,1,-15,-15,0,-10
3,9.88030,34.22228,DCA,19,41.03738,81,1,33.52233,0,45,45,1,51
4,11.70175,29.06825,MSP,22,42.37835,85,1,33.52233,0,15,15,1,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,6.95741,29.17117,ATL,20,42.09990,152,1,33.52233,0,-15,-15,0,-11
249996,9.88030,35.70655,DFW,21,42.26300,186,1,33.52233,1,45,45,1,39
249997,9.88030,32.95197,CLT,9,27.26231,108,1,33.52233,0,-15,-15,0,-5
249998,9.88030,37.40948,ORF,22,42.37835,68,1,33.52233,0,15,15,1,13


In [53]:
#df=df_stage.dropna()
### or 
df1=df_stage.dropna()
df1


,carrier_v,dest_v,dest,crs_arr_hh24,hh24_v,crs_elapsed_time,month,month_v,haul_type,arr_delay_min_gy,arr_delay_min_bin,arr_delay_ind,arr_delay
0,8.33723,44.79845,EWR,11,29.34927,82,1,33.52233,0,-15,-15,0,-26
1,5.37583,35.65979,STS,9,27.26231,102,1,33.52233,0,15,15,1,9
2,11.70175,29.17117,ATL,9,27.26231,185,1,33.52233,1,-15,-15,0,-10
3,9.88030,34.22228,DCA,19,41.03738,81,1,33.52233,0,45,45,1,51
4,11.70175,29.06825,MSP,22,42.37835,85,1,33.52233,0,15,15,1,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,6.95741,29.17117,ATL,20,42.09990,152,1,33.52233,0,-15,-15,0,-11
249996,9.88030,35.70655,DFW,21,42.26300,186,1,33.52233,1,45,45,1,39
249997,9.88030,32.95197,CLT,9,27.26231,108,1,33.52233,0,-15,-15,0,-5
249998,9.88030,37.40948,ORF,22,42.37835,68,1,33.52233,0,15,15,1,13


In [54]:
df=df1[['carrier_v','dest_v','hh24_v', 'month_v', 'arr_delay' ]]
train_df, test_df = train_test_split(df, test_size=0.2, random_state=123)
X_train, y_train = train_df.drop(columns=["arr_delay"], axis=1), train_df["arr_delay"]
X_test, y_test = test_df.drop(columns=["arr_delay"], axis=1), test_df["arr_delay"]
lr = LogisticRegression()
lr.fit(X_train, y_train)



/Users/pelecheung/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [55]:
X_train.columns

Index(['carrier_v', 'dest_v', 'hh24_v', 'month_v'], dtype='object')

In [56]:
print("Model weights: %s"%(lr.coef_)) # these are weights
print("Model intercept: %s"%(lr.intercept_)) # this is the bias term
data = {'features': X_train.columns, 'coefficients':lr.coef_[0]}
pd.DataFrame(data)

Model weights: [[-0.01389972 -0.03693994 -0.02113097 -0.02950245]
 [-0.01847325 -0.0494556  -0.02855329 -0.04107126]
 [-0.01617769 -0.04016804 -0.02540858 -0.03145063]
 [-0.01799497 -0.04746388 -0.03055826 -0.04601342]
 [-0.01436723 -0.03456058 -0.0337393  -0.02782022]
 [-0.01403418 -0.03431256 -0.01030922 -0.02348906]
 [-0.02776182 -0.04165024  0.00275578 -0.03194296]
 [-0.02074305 -0.02916469 -0.02024254 -0.025276  ]
 [-0.01091023 -0.01990229 -0.03332015 -0.02040289]
 [-0.0220049  -0.0255472  -0.02377567 -0.02185336]
 [-0.03056306 -0.02403812 -0.00553019 -0.01718667]
 [-0.02099897 -0.0181687  -0.01407882 -0.02377646]
 [-0.01405858 -0.02950867 -0.00031486 -0.02109207]
 [-0.05088633 -0.01570153 -0.02246322  0.0027192 ]
 [-0.03084656 -0.01367125 -0.01842233 -0.00255412]
 [-0.03187282  0.01643237 -0.05269195  0.00265581]
 [-0.05094702 -0.0205307  -0.00420789  0.00176896]
 [-0.06298839  0.00214567 -0.0252335   0.01061417]
 [-0.0694716  -0.00508337 -0.01307022  0.00799732]
 [-0.06839856 -0

,features,coefficients
0,carrier_v,-0.013900
1,dest_v,-0.036940
2,hh24_v,-0.021131
3,month_v,-0.029502


In [57]:
example = X_test.iloc[0,:]
example.tolist()

[17.76531, 34.34729, 34.27183, 33.52233]

In [58]:
lr.intercept_

array([-8.80552970e-04, -1.22560982e-03, -9.38647732e-04, -1.37303153e-03,
       -8.30223005e-04, -7.00973745e-04, -9.53172574e-04, -7.54264208e-04,
       -6.08887024e-04, -6.52159548e-04, -5.12911744e-04, -7.09508331e-04,
       -6.29385064e-04,  8.10087047e-05, -7.63132843e-05,  7.91266390e-05,
        5.26798135e-05,  3.16581411e-04,  2.38514355e-04,  2.19157576e-04,
        5.16544671e-04,  1.24147691e-03,  6.50770368e-04,  1.16385984e-03,
        1.56246789e-03,  2.23894599e-03,  1.92908091e-03,  2.70008542e-03,
        2.31125352e-03,  2.49629869e-03,  3.85585411e-03,  3.41155815e-03,
        3.42544661e-03,  4.29464406e-03,  3.46277298e-03,  3.84200392e-03,
        4.78267621e-03,  4.26606671e-03,  4.41745001e-03,  4.61513202e-03,
        4.38085877e-03,  4.17026145e-03,  4.00565276e-03,  4.22195918e-03,
        3.90891328e-03,  4.06992186e-03,  3.62367941e-03,  3.47972523e-03,
        3.89240904e-03,  3.75794751e-03,  3.60497308e-03,  3.48581369e-03,
        3.41579806e-03,  

In [59]:
lr.classes_

array([-60, -59, -58, -57, -56, -55, -54, -53, -52, -51, -50, -49, -48,
       -47, -46, -45, -44, -43, -42, -41, -40, -39, -38, -37, -36, -35,
       -34, -33, -32, -31, -30, -29, -28, -27, -26, -25, -24, -23, -22,
       -21, -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,
        -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,
         5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89])

## Start preidct

In [60]:
database = "mtpj.db"

conn = sqlite3.connect(database)

## For prove of concept use Flights_slim instead of Flights
sql_in="""SELECT cf.delay_pct as carrier_v, 
          --of.delay_pct as origin_v,
          df.delay_pct as dest_v,
          df.airport as dest,
          CAST(fl.crs_arr_hh24  as interger) crs_arr_hh24,
          hf.delay_pct  as                    hh24_v,
          --CAST(crs_elapsed_time as interger) crs_elapsed_time,
          CAST(fl.fl_month      as interger) month,
          mf.delay_pct  as                   month_v
          --CAST(haul_type        as interger) haul_type
          FROM  flights_test_7days fl, CARRIER_DELAY_F cf, AIRPORT_F df, AIRPORT_F of,
                HH24_F      hf, MONTH_F mf
          WHERE fl.mkt_unique_carrier = cf.mkt_unique_carrier
          AND   fl.dest               = df.airport
          AND   fl.origin             = of.airport
          AND   fl.crs_arr_hh24       = hf.crs_arr_hh24
          AND   fl.fl_month           = mf.fl_month
          ORDER BY RANDOM()
          LIMIT 160000;"""

df_stage = pd.read_sql(sql_in, con=conn)
conn.close()
df_stage
##df_stage=pd.read_csv('flights_f1.csv')

,carrier_v,dest_v,dest,crs_arr_hh24,hh24_v,month,month_v
0,8.33723,34.66992,DEN,18,40.37584,1,33.52233
1,9.88030,37.03443,PVD,23,41.87090,1,33.52233
2,9.88030,37.71784,BOS,19,41.03738,1,33.52233
3,11.70175,37.82796,TUS,17,38.06039,1,33.52233
4,6.95741,39.22392,SEA,9,27.26231,1,33.52233
...,...,...,...,...,...,...,...
150618,8.33723,36.48137,ORD,6,27.47973,1,33.52233
150619,6.95741,33.35622,SJC,19,41.03738,1,33.52233
150620,17.76531,36.54618,JAX,11,29.34927,1,33.52233
150621,11.70175,35.61552,LAS,18,40.37584,1,33.52233


In [61]:
df1=df_stage.dropna()
df1

,carrier_v,dest_v,dest,crs_arr_hh24,hh24_v,month,month_v
0,8.33723,34.66992,DEN,18,40.37584,1,33.52233
1,9.88030,37.03443,PVD,23,41.87090,1,33.52233
2,9.88030,37.71784,BOS,19,41.03738,1,33.52233
3,11.70175,37.82796,TUS,17,38.06039,1,33.52233
4,6.95741,39.22392,SEA,9,27.26231,1,33.52233
...,...,...,...,...,...,...,...
150618,8.33723,36.48137,ORD,6,27.47973,1,33.52233
150619,6.95741,33.35622,SJC,19,41.03738,1,33.52233
150620,17.76531,36.54618,JAX,11,29.34927,1,33.52233
150621,11.70175,35.61552,LAS,18,40.37584,1,33.52233


In [62]:
dfp=df1[['carrier_v','dest_v','hh24_v', 'month_v' ]]
dfp


,carrier_v,dest_v,hh24_v,month_v
0,8.33723,34.66992,40.37584,33.52233
1,9.88030,37.03443,41.87090,33.52233
2,9.88030,37.71784,41.03738,33.52233
3,11.70175,37.82796,38.06039,33.52233
4,6.95741,39.22392,27.26231,33.52233
...,...,...,...,...
150618,8.33723,36.48137,27.47973,33.52233
150619,6.95741,33.35622,41.03738,33.52233
150620,17.76531,36.54618,29.34927,33.52233
150621,11.70175,35.61552,40.37584,33.52233


In [66]:
example = dfp.iloc[0,:]
example.tolist()
#example.describe

[8.33723, 34.66992, 40.37584, 33.52233]

In [67]:
lr.predict([example])

array([-14])

In [68]:
lr.predict_proba([example])

array([[1.16260027e-04, 3.64499841e-05, 8.03861181e-05, 3.06369970e-05,
        7.99814099e-05, 2.40928196e-04, 2.12621797e-04, 1.71746515e-04,
        1.78503624e-04, 1.87373081e-04, 4.49135253e-04, 3.38422391e-04,
        4.61647244e-04, 4.98255453e-04, 6.23159037e-04, 5.23781958e-04,
        8.52669202e-04, 9.74590381e-04, 1.07560285e-03, 1.30104834e-03,
        1.40066083e-03, 1.87277064e-03, 2.29076016e-03, 2.54133151e-03,
        3.16278059e-03, 3.14939850e-03, 3.79147781e-03, 4.25341816e-03,
        4.66590829e-03, 6.11819125e-03, 6.65841982e-03, 7.96202944e-03,
        8.58581649e-03, 9.52115961e-03, 1.11818084e-02, 1.11963291e-02,
        1.21832312e-02, 1.42028542e-02, 1.54325862e-02, 1.63275524e-02,
        1.72671421e-02, 1.99125169e-02, 1.98023738e-02, 2.24700260e-02,
        2.43991175e-02, 2.39554793e-02, 2.63239871e-02, 2.58374407e-02,
        2.57108610e-02, 2.48218739e-02, 2.48967135e-02, 2.62245199e-02,
        2.56733184e-02, 2.48975656e-02, 2.30681312e-02, 2.321435

In [69]:
data_dict = {"predicted": lr.predict(dfp).tolist()}
pd.DataFrame(data_dict)
out_file=pd.DataFrame(data_dict)
out_file

,predicted
0,-14
1,-9
2,-9
3,-9
4,-9
...,...
150618,-9
150619,-14
150620,-7
150621,-9


In [70]:
out_file.to_csv('predict.csv')